### Your first hypervelocity star catalogue

`speedystar` is a python package which will allow you to generate, evolve, propagate and perform mock observations of single stars ejected at high velocities.

### Getting Started

Make sure this notebook is in the parent directory of `speedystar/`. The package itself is not large, but performing mock observations requires downloading a Milky Way dust map which by default is ~500 MB. This free space is assumed to be in the current working directory by default, but does not have to be (see below).

We currently do not recommend installing `speedystar` globally using pip since that prevents you from editing the source code directly.

`speedystar` uses a number of Python packages. Some of these which might not already be included in your Python distribution are astropy, galpy, mwdust and pygaia. Simply run this notebook and conda or pip install any package that doesn't exist.

More accurate treatments of the Gaia selection functions and astrometric spread function also require the Python packages scanninglaw and gaiaunlimited. These packages currently do not support Windows since they rely on the healpy package, but can be installed using Windows Subsystem for Linux (WSL).

### Documentation

help(speedystar.starsample) will display the names and descriptions of every method in `speedystar`, as well as descriptions of common column names.

### Outputs

`speedystar` outputs are .fits tables containing useful info for each mock star as well as metadata describing the assumptions that go into the sample. They can be accessed using astropy.table or with [Topcat](https://www.star.bris.ac.uk/~mbt/topcat/) or however else you'd like to read them.


In [1]:
#Import what you need
import numpy as np
import os
#os.chdir('/mnt/c/Users/frase/')
from speedystar import starsample
from speedystar.eject import Hills
from speedystar.eject import HillsFromCatalog
from speedystar.utils.mwpotential import MWPotential
import astropy.units as u
from galpy import potential
import mwdust
from tqdm import tqdm
import matplotlib.pyplot as plt
from astropy.table import Table
#Print a lot of documentation
#help(starsample)

/Users/mncavieres/opt/anaconda3/envs/speedytest2/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(



### Ejecting a sample

We're now ready to eject some stars!

The 'typical' way to eject hypervelocity stars is via the Hills Mechanism, in which a stellar binary approaches Sgr A* on a very low-angular momentum orbit. At a particular distance from Sgr A*, the tidal forces across the binary surpass the gravitational binding energy of the binary itself and the binary is separated. One star is captured in orbit around Sgr A* and the other is flung away at a velocities up to several thousand km/s. Stars ejected above the escape velocity of the Galaxy are hypervelocity stars (HVSs) -- they will eventually escape the Galaxy entirely. Note that there will also be a population of stars ejected slowly as well, which will not escape the inner Galaxy and might survive to interact again with Sgr A*.

<p align="center">
<img src ="https://www.universetoday.com/wp-content/uploads/2023/01/hills-mechanism.jpg" width="50%" height="50%">
</p>

With `speedystar` you first generate a sample of stars at the moment of ejection. The number of stars and their masses/metallicities/velocities/flight times/evolutionary stages depend a lot on assumptions you make about stars and binaries in the Galactic Centre, see [Evans+2022](https://ui.adsabs.harvard.edu/abs/2022MNRAS.512.2350E/abstract) for more details.


## Build a simple catalog

In [2]:
simulated_catalogs_path = '/Users/mncavieres/Documents/2024-2/HVS/Data/speedystar_catalogs/from_MIST'

In [3]:
# load MIST isochrone simulated catalog
catalog_mist = Table.read('/Users/mncavieres/Documents/2024-2/HVS/Data/SFH_sampling_catalogs/MIST_photometry/sample_NSC_with_photometry.fits')

In [4]:
# subsample the catalog
catalog_mist = catalog_mist[np.random.choice(len(catalog_mist), 1000, replace=False)]

In [25]:
catalog_mist

mass,age,flight_time,G,BP,RP,Teff,logg,radius,logL,feh,eep
solMass,Gyr,Gyr,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.856900640808,5.12323612064,7.1058775019,5.58473787645,6.04255122309,4.97310578769,5056.77056849,4.4803234534,0.881829322814,-0.33961499994,0.27386280519,383.803195953
0.850508747207,4.84098806365,0.197818745916,6.01789734432,6.52321771513,5.36700519146,4871.74525027,4.56780603693,0.794326075928,-0.494841461297,0.320983607553,333.964862823
0.828650007448,10.8640016821,1.03357909892,5.87494044042,6.36822902415,5.23359806189,4908.23741125,4.51522973357,0.833033014818,-0.441211352397,0.279681320923,369.187545776
0.96547920366,6.10957031324,5.21196716612,4.65565311531,5.03464711603,4.11469068486,5451.16196009,4.31732236798,1.12917746834,0.00581104452721,0.261274901996,421.874771118
1.28839675007,2.1915532961,0.158019880148,3.59060133269,3.85639050876,3.16755789616,6177.17043738,4.25331055098,1.40444549304,0.412380436933,0.290513327727,350.39686203
1.0911028146,4.69381182227,7.9427677897,--,--,--,--,--,--,--,--,--
1.85028016343,1.41474762848,4.03327526347,--,--,--,--,--,--,--,--,--
0.821502453196,3.61057111723,8.62338786862,5.92640316655,6.42707387599,5.27899637443,4878.72442336,4.51832845772,0.826480534573,-0.458558564447,0.278617588689,368.917980194


In [15]:
ejectionmodel = HillsFromCatalog(
    catalog=catalog_mist,
    rate=1e-4/u.yr,  # Same rate parameter as in the original model
    kappa=0,         # Log-slope of the IMF (not used here, but must be passed)
    amuseflag=True,  # Keep the flag for consistency
    Met=0.25         # Metallicity (not used here, but for interface consistency)
)
mysample = starsample(ejectionmodel, name = 'MIST')

In [6]:
mysample.saveall('/Users/mncavieres/Documents/2024-2/HVS/Data/speedystar_catalogs/from_MIST/test2.fits')

### Propagating the sample

The 'ejection sample' consists of a population of stars freshly ejected from the centre of the Milky Way. Next we have to propagate them through the Galaxy (each at its own assigned flight time) to find out where they will be at the present day. To do this we will have to assume a potential for the Galaxy. 

<p align="center">
<img src ="https://cdn.sci.news/images/enlarge4/image_5003e-Hypervelocity-Stars.jpg" width="50%" height="50%">
</p>


In [8]:
propagated_catalogs_path = simulated_catalogs_path
#simulated_catalogs_path = '/Users/mncavieres/Documents/2024-2/HVS/speedystar/simulated_catalogs/top_heavy'

# Assume a Galactic potential
default_potential = MWPotential()
#Ensure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(default_potential)

#Propagate sample. Positions / velocities are returned in Cartesian (x/y/z/v_x/v_y/v_z) and Galactic (l/b/dist/pm_l/pm_b/v_radial) and equatorial (ra/dec/dist/pm_ra/pm_dec/v_radial) coordinates
mysample.propagate(potential = default_potential)

Propagating...: 100%|██████████| 1000/1000 [00:34<00:00, 28.72it/s]


In [9]:
#Save propagated sample
mysample.saveall('/Users/mncavieres/Documents/2024-2/HVS/Data/speedystar_catalogs/from_MIST/test2_propagated.fits')

In [22]:

mysample = starsample('/Users/mncavieres/Documents/2024-2/HVS/Data/speedystar_catalogs/from_MIST/test2_propagated.fits')

### Subsetting the catalogue

So far we've ejected and propagated everything, including fast and slow stars. If we're only interested in the hypervelocity stars, we can use boolean indexing on a speedystar object.

In [20]:
fast = mysample.GCv > mysample.Vesc

print('Size of the entire sample: '+str(mysample.size))

mysample = mysample[fast]
print('Size of the fast sample: '+str(mysample.size))

Size of the entire sample: 1000
Size of the fast sample: 360


### Mock observations of the sample

We next have to figure out how bright each of the stars is, otherwise we don't know which of the stars would be detectable today or in the near future. `speedystar` is able to calculate the apparent magnitudes of the sample in a variety of photometric bassbands (e.g. Gaia G/G_BP/G_RP/G_RVS, Johnson-Cousins UBVRI, SDSS/LSST ugriz, VISTA JHK) depending on the mass, temperature and radius of the stars along with their distance and position on the sky.



In [15]:
mysample.met = np.array([0.2]*1000)

In [23]:

# # Select only HVSs that are fast
# fast = mysample.GCv > mysample.Vesc
# mysample = mysample[fast]

#Assign the dust map. Will be downloaded if it doesn't already exist in the working directory or where you've
#   specified above
mysample.dust = mwdust.Combined15()

#Get mock apparent magnitudes . By default magnitudes are computed in the Johnson-Cousins V and I bands 
#   and the Gaia G, G_RP, G_BP and G_RVS bands.
#   By default this also returns Gaia astrometric and radial velocity errors assuming Gaia DR4 precision
mysample.photometry()

#Save the sample with mock photometry
mysample.save('/Users/mncavieres/Documents/2024-2/HVS/Data/speedystar_catalogs/from_MIST/test2_propagated_phot.fits')

Calculating magnitudes:  33%|███▎      | 2/6 [00:00<00:00,  4.72it/s]

/Users/mncavieres/Documents/2024-2/HVS/speedystar_v2/speedystar/utils/MIST_photometry.py:815: RuntimeWarning: invalid value encountered in log10
  VMag0 = MbolSun - 2.5*np.log10(Lum.value) - BC



Calculating magnitudes:  50%|█████     | 3/6 [00:00<00:00,  3.45it/s]

/Users/mncavieres/Documents/2024-2/HVS/speedystar_v2/speedystar/utils/MIST_photometry.py:830: RuntimeWarning: invalid value encountered in log10
  IMag0 = MbolSun - 2.5*np.log10(Lum.value) - BC



Calculating magnitudes:  67%|██████▋   | 4/6 [00:01<00:00,  2.93it/s]

/Users/mncavieres/Documents/2024-2/HVS/speedystar_v2/speedystar/utils/MIST_photometry.py:838: RuntimeWarning: invalid value encountered in log10
  GMag0 = MbolSun - 2.5*np.log10(Lum.value) - BC



Calculating magnitudes:  83%|████████▎ | 5/6 [00:01<00:00,  2.77it/s]

/Users/mncavieres/Documents/2024-2/HVS/speedystar_v2/speedystar/utils/MIST_photometry.py:846: RuntimeWarning: invalid value encountered in log10
  RPMag0 = MbolSun - 2.5*np.log10(Lum.value) - BC



Calculating magnitudes: 100%|██████████| 6/6 [00:04<00:00,  1.39it/s]

/Users/mncavieres/Documents/2024-2/HVS/speedystar_v2/speedystar/utils/MIST_photometry.py:854: RuntimeWarning: invalid value encountered in log10
  BPMag0 = MbolSun - 2.5*np.log10(Lum.value) - BC



In [14]:
#Load a pre-existing propagated sample, if needed
mysample = starsample('./cat_propagated.fits')

#Magnitudes are exctincted by Milky Way dust along the line of sight. Need to assign a dust map. 
#Dust map(s) must be downloaded if they do not already exist. They can be rather large, ~500 MB.

#Uncomment this line and fill out a path if the dust map is located somewhere other than the working
#   directory, or you want it downloaded somewhere other than the working directory
#mysample.config_dust('/path/to/where/you/want/the/dust/map')

#Assign the dust map. Will be downloaded if it doesn't already exist in the working directory or where you've
#   specified above
mysample.dust = mwdust.Combined15()

#Get mock apparent magnitudes . By default magnitudes are computed in the Johnson-Cousins V and I bands 
#   and the Gaia G, G_RP, G_BP and G_RVS bands.
#   By default this also returns Gaia astrometric and radial velocity errors assuming Gaia DR4 precision
mysample.photometry()

#Save the sample with mock photometry
mysample.save('./cat_photometry.fits')

Calculating magnitudes: 100%|██████████| 6/6 [00:02<00:00,  2.16it/s]


### Gaia detectability
 Finally, let's determine which stars would be detectable in Gaia Data Release 3.
 
 Gaia magnitude cuts can be performed using mysample[...], but they're also implemented directly in `speedystar.subsample` for simplicity, along with some other common cuts. These cuts will also automatically calculate DR3 mock Gaia errors.


In [15]:
detectable_dr3_path = '/Users/mncavieres/Documents/2024-2/HVS/speedystar/simulated_catalogs/detectable_DR3'

for catalog in tqdm.tqdm(os.listdir(photometric_catalogs_path)):
    if not catalog.endswith('.fits'):
        pass
    # Load photometric sample
    mysample = starsample(os.path.join(photometric_catalogs_path, catalog))

    #Determine which stars would be in principle detectable in Gaia DR3 and recalculate errors
    mysample.subsample('Gaia_DR3')

    #Save the cut sample
    mysample.save(os.path.join(detectable_dr3_path, catalog))

100%|██████████| 100/100 [00:10<00:00,  9.95it/s]


In [15]:
#Load a pre-existing sample with photometry, if needed
mysample = starsample('./cat_photometry.fits')

#Determine which stars would be in principle detectable in Gaia DR3 and recalculate errors
mysample.subsample('Gaia_DR3')

#Save the cut sample
mysample.save('./cat_gaiaDR3.fits')
print('Number of stars in Gaia DR3: '+str(mysample.size))

#Determine which stars would be in principle detectable in the subsample of Gaia DR4 with
#   measured radial velocities.
mysample.subsample('Gaia_6D_DR4')

#Save the cut sample
mysample.save('./cat_gaiaDR4_6D.fits')
print('Number of stars in Gaia DR4 with radial velocity: '+str(mysample.size))


Number of stars in Gaia DR3: 95
Number of stars in Gaia DR4 with radial velocity: 4


In [17]:
#Load a pre-existing sample with photometry, if needed
mysample = starsample('./cat_photometry.fits')
mysample.subsample('Gaia_DR4')

#Save the cut sample
mysample.save('./cat_gaiaDR4.fits')
print('Number of stars in Gaia DR4: '+str(mysample.size))

Number of stars in Gaia DR4: 95


In [20]:
#Load a pre-existing sample with photometry, if needed
mysample = starsample('./cat_photometry.fits')
mysample.subsample('Gaia_6D_DR4')
print('Number of stars in Gaia DR4: '+str(mysample.size))

Number of stars in Gaia DR4: 4
